# Analiza - sprawdzenie bardzo prostej strategii opartej jedynie na dwóch formacjach świecowych + "zasada 11/20" + dodatkowo RSI

<h3>
<center>
Robert Nowotniak <rnowotniak@gmail.com><br/>
w oparciu o metodykę Fraktal Trader Moduł 1, formacje świecowe
</center>
</h3>

* [Eksperyment 1) Analiza młotków - EURUSD 1h](#eksperyment1)

* [Eksperyment 2) Spadające gwiazdy - Dow Jones 1h](#eksperyment2)

* [Eksperyment 3) Analiza zbiorcza dla wszystkich walorów - Młotki](#eksperyment3)

* [Eksperyment 3) - Podsumowanie zbiorcze](#eksperyment3summary)
 
* [Eksperyment 4) Analiza zbiorcza dla wszystkich walorów - Spadające gwiazdy](#eksperyment4)

* [Eksperyment 4) - Podsumowanie zbiorcze](#eksperyment4summary)

In [25]:
import pandas as pd
import numpy as np
import pandas_datareader.data as web
import talib
import talib.abstract as ta
from datetime import datetime
import matplotlib.pyplot as plt
from IPython.core.display import HTML
import seaborn as sns
import glob, os.path
sns.set_style('whitegrid')
%matplotlib inline

In [7]:
def ohlc(r):
    return r['open'], r['high'], r['low'], r['close']

## Wczytanie specyfikacji wszystkich instrumentów

Specyfikacja instrumentów wygenerowana w MetaTraderze skryptem MQL allmarketdata.ex4

In [3]:
specs = pd.read_csv('instruments.csv', sep=';', index_col=0)

# XXX - nie da się tego wyciągnąć bezpośrednio ze specyfikacji ??
def setPipSize(x):
    return float(x['Tick Size']) # XXX - uproszczenie, nie jestem pewien, czy to jest prawidłowe
                                 #       ale chyba nie ma znaczenia
    if x['Pr. Mode'] == 'Forex':
        if x['Symbol'].endswith('JPY'):
            return 0.01
        # elif x['Symbol'].endswith('HUF'):  # XXX Hungarian Forint ???
        return 0.0001
    return 1
specs['Pip'] = specs.apply(setPipSize, axis=1)

### Wczytanie historycznych danych wszystkich walorów do klasy Instrument

In [35]:
class Instrument:
    def __init__(self, name):
        self.name = name
        self.spec = None
        self.results = None
        self.balance = 0  # +/- gain value
        self.stats = {} # PROFIT: ...,  LOSS: ...
        self.success_rate = None
        self.data = pd.read_csv('data/%s60.csv' % name, index_col = False,
                   names=['day', 'hour', 'open', 'high', 'low', 'close', 'volume'],
                               dtype={'open':np.float64,'high':np.float64,
                                     'low':np.float64,'close':np.float64,'volume':np.float64}
                               )
        self.data['date'] = pd.DatetimeIndex(self.data.day + ' ' + self.data.hour)
        del self.data['day']
        del self.data['hour']
        self.data.set_index('date', inplace = True)
        
        # Calculate RSI14 for this instrument
        self.rsi = ta.RSI(self.data, 14)
    
    def __str__(self):
        s = self.name
        if self.data is not None:
            s = s + ' ' + str(self.data.shape)
        return s

INSTRUMENTS = {}        
        
for f in glob.glob('data/*60.csv'):
    name = os.path.basename(f).replace('60.csv', '')
    print(name)
    inst = Instrument(name)
    inst.spec = specs.ix[name]
    INSTRUMENTS[name] = inst
 

USDCHF
EURDKK
EURUSD
EURPLN
EURNOK
COCOA-SEP17
GER30Cash
EURCHF
BRENT-AUG17
POL20Cash
GBPCAD
SUGAR-JUL17
CADJPY
GOLD
EURRUB
SILVER
AUDCAD
US30Cash
EURSEK
AUDJPY
AUDCHF
EURAUD
USDCAD
EURHKD
AUDNZD


In [40]:
INSTRUMENTS['EURRUB'].rsi['2017/06/16 10:00']

67.19948864159953

## Zasada 11 / 20 - porównywanie rozmiaru świecy z 20 poprzednimi

In [8]:
def zasada1120(inst, t):
    
    BACKSIZE = 20         # XXX Wyciągnąć gdzieś do konfiguracji
    CANDLES_REQUIRED = 11 # XXX
    
    t = pd.Timestamp(t)
    i = inst.data.index.get_loc(t, 'nearest')
    o, h, l, c = ohlc(inst.data.iloc[i])
    size = h - l
   
    j = 1
    smaller_candles = 0
    while j <= BACKSIZE and i - j >= 0 and smaller_candles < CANDLES_REQUIRED:
        o, h, l, c = ohlc(inst.data.iloc[i - j])
        if size > h - l:
            smaller_candles += 1
        j += 1
    if smaller_candles >= CANDLES_REQUIRED:
        return True
    
    return False
    
zasada1120(INSTRUMENTS['US30Cash'], '2017/06/20 22:00')

True

## Sprawdzanie, czy dany sygnał był prawidłowy(czy zarobił) -- B. WAŻNY FRAGMENT
### (ta funkcja docelowo byłaby jednym z głównych elementów pełnego symulatora strategii)

In [9]:
# returns:
#   (PROFIT/LOSS, gain value, nswaps, sp, tp, size)
def sprawdz(inst, t, buysell = 'buy'):
    t = pd.Timestamp(t)
    i = inst.data.index.get_loc(t, 'nearest')
    o, h, l, c = ohlc(inst.data.iloc[i])
    size = h - l
    
    spread = 3 # XXX  US30
    spread = 0.0003 # XXX EURUSD
    spread = float(inst.spec['Spread']) / 10**int(inst.spec['Digits'])  # XXX
    #print(spread)
    PROFIT_TO_RISK = 3  # XXX Wyciagnac do konfiguracji
    
    if buysell == 'buy':
        sl = l - inst.spec['Pip'] # 1 punkt pod formacja
        risk = c - sl + spread # -> mozliwa strata = roznica poziomow + spread
        tp = c + spread + risk * PROFIT_TO_RISK # -> mozliwy zysk = roznica poziomow - spread = risk*PTR
    elif buysell == 'sell':
        sl = h + inst.spec['Pip'] + spread # 1 punkt nad formacja + spread (chyba z zalozenia ze spada szybciej)
        risk = sl - c + spread # mozliwa strata = roznica poziomow + spread
        tp = c - risk * PROFIT_TO_RISK - spread  # -> mozliwy zysk = roznica poziomow - spread = risk*PTR
    else:
        raise Exception('buysell')
    # print(sl, tp)
    
    before_midnght = inst.data.index[i].hour > 12
    nswaps = 0
    while i < inst.data.shape[0] - 1:
        i += 1
        o, h, l, c = ohlc(inst.data.iloc[i])
        
        if before_midnght and inst.data.index[i].hour < 12:
            # print('SWAP')
            nswaps += 1
        before_midnght = inst.data.index[i].hour > 12
        
        result = None
        if (buysell == 'buy' and h >= tp) or (buysell == 'sell' and l <= tp):
            # print(inst.data.index[i])
            result = 'PROFIT'
        if (buysell == 'buy' and l <= sl) or (buysell == 'sell' and h >= sl):
            # print(inst.data.index[i])
            if result == 'PROFIT':
                return 'Unknown'
            result = 'LOSS'
        if result == 'PROFIT':
            return result, PROFIT_TO_RISK * risk, nswaps, sl, tp, size #, zasada1120(inst, t)
        if result == 'LOSS':
            return result, -risk, nswaps, sl, tp, size #, zasada1120(inst, t)

t = '2017/06/22 01:00'
print(sprawdz(INSTRUMENTS['EURUSD'], t))

('LOSS', -0.00033000000000009454, 0, 1.1164399999999999, 1.1177600000000003, 0.00029000000000012349)


## Definicja wybranych formacji świecowych - funkcje wykrywające

In [10]:
def jestMlotek(row):
    o, h, l, c = ohlc(row)
    if c > o and c - o <= 0.4 * (h - l):
        if h - c <= 0.14 * (h - l):
            return True
    if c < o and o - c <= 0.4 * (h - l):
        if h - o <= 0.14 * (h - l):
            return True    
    return False

def jestSpadajacaGwiazda(row):
    o, h, l, c = ohlc(row)
    if c > o and c - o <= 0.4 * (h - l):
        if o - l <= 0.14 * (h - l):
            return True
    if c < o and o - c <= 0.4 * (h - l):
        if c - l <= 0.14 * (h - l):
            return True    
    return False

# ...
# Tutaj dalej docelowo powinny być zaimplementowane inne formacje (przenikania, zasłona ciemnej chmury itp)
# ...

<a id="eksperyment1"></a>
## Eksperyment 1) Analiza młotków - EURUSD 1h

In [46]:
inst = INSTRUMENTS['EURUSD']

stats = {}
balance = 0.
for idx, row in inst.data['2017/01/01':'2017/07/01'].iterrows():
    if jestMlotek(row) and zasada1120(inst,idx) and inst.rsi[idx] <= 30:
        res = sprawdz(inst, idx)
        if not res:
            print(idx, ' ', res)
            continue        
        print(idx, ' ', '%6s'%res[0], ' ', end='')
        for _ in res[1:]:
            print('%.5f'%_, ' ', end='')
        print()
        if type(res) == tuple:
            balance += res[1] # XXX tu ewentualnie uwzgledniac swap * nswaps
            if res[0] not in stats:
                stats[res[0]] = 0
            stats[res[0]] += 1
print(balance)
print(stats)
if 'PROFIT' in stats:
    print("Success rate: %.2f%%" % (100.0*stats['PROFIT']/ sum(stats.values())))

2017-02-07 10:00:00     LOSS  -0.00145  0.00000  1.06606  1.07186  0.00226  
2017-02-10 16:00:00     LOSS  -0.00163  1.00000  1.06069  1.06721  0.00167  
2017-02-22 12:00:00     LOSS  -0.00139  0.00000  1.04933  1.05489  0.00164  
2017-06-15 19:00:00     LOSS  -0.00110  1.00000  1.11384  1.11824  0.00121  
-0.00557
{'LOSS': 4}


<a id="eksperyment2"></a>
## Eksperyment 2) Spadające gwiazdy - Dow Jones 1h

In [49]:
inst = INSTRUMENTS['US30Cash']

balance = 0.
stats = {}
for idx, row in inst.data['2017/01/01':'2017/07/01'].iterrows():
    if jestSpadajacaGwiazda(row) and zasada1120(inst,idx) and inst.rsi[idx] >= 70:
        res = sprawdz(inst, idx, 'sell') # XXX <- SELL
        if not res:
            print(idx, ' ', res)
            continue
        print(idx, ' ', '%6s'%res[0], ' ', end='')
        for _ in res[1:]:
            print('%.5f'%_, ' ', end='')
        print()
        if type(res) == tuple:
            balance += res[1] # XXX tu ewentualnie uwzgledniac swap * nswaps
            if res[0] not in stats:
                stats[res[0]] = 0
            stats[res[0]] += 1
print(balance)  
print(stats)
if 'PROFIT' in stats:
    print("Success rate: %.2f%%" % (100.0*stats['PROFIT']/ sum(stats.values())))

2017-04-24 02:00:00     LOSS  -30.61000  0.00000  20741.81000  20619.37000  26.50000  
2017-04-25 22:00:00     LOSS  -31.11000  1.00000  21025.31000  20900.87000  28.00000  
2017-06-19 20:00:00     LOSS  -21.61000  0.00000  21522.01000  21435.57000  20.50000  
-83.33
{'LOSS': 3}


<a id="eksperyment3"></a>
## Eksperyment 3) Analiza zbiorcza dla wszystkich walorów - Młotki

In [43]:
%%time

for name in INSTRUMENTS:
    print(name)
    inst = INSTRUMENTS[name]
    inst.results = pd.DataFrame( columns = range(7) )

    stats = {}
    balance = 0.
    for idx, row in inst.data['2017/01/01':].iterrows():  # XXX Month
        if jestMlotek(row) and zasada1120(inst,idx)  and inst.rsi[idx] <= 30: # XXX jestMlotek
            res = sprawdz(inst, idx) # XXX buy/sell
            if not res or type(res) == str:
                continue        
            inst.results = inst.results.append([ (idx,) + res ])
            if type(res) == tuple:
                balance += res[1] # XXX tu ewentualnie uwzgledniac swap * nswaps
                if res[0] not in stats:
                    stats[res[0]] = 0
                stats[res[0]] += 1
    inst.results.columns = ['date', 'PL', 'gv', 'nswaps', 'sl', 'tp', 'size']
    inst.results.set_index('date', inplace=True)
    inst.balance = balance
    inst.stats = stats
    inst.success_rate = 0
    if 'PROFIT' in stats:
        inst.success_rate = stats['PROFIT']/ sum(stats.values())

CADJPY
USDCAD
US30Cash
GBPCAD
USDCHF
BRENT-AUG17
EURAUD
EURCHF
EURNOK
EURSEK
AUDCAD
EURPLN
AUDJPY
AUDNZD
POL20Cash
SILVER
EURUSD
SUGAR-JUL17
GER30Cash
AUDCHF
EURRUB
EURDKK
COCOA-SEP17
GOLD
EURHKD
CPU times: user 26.2 s, sys: 23 ms, total: 26.2 s
Wall time: 26.2 s


In [50]:
for iname in sorted(INSTRUMENTS.keys()):
    inst = INSTRUMENTS[iname]
    if inst.results is not None:
        display(HTML('<h3>%s</h3>' % iname))
        display(HTML('<b>Balance: </b> %f &nbsp;&nbsp;&nbsp;&nbsp; <b>Trades:</b>%s' % (inst.balance, inst.stats)))
        display(HTML('<b>Success rate: </b> %.2f%%' % (100. * inst.success_rate)))
        print("One month details:")
        #display(inst.results['2017/06/01':])        
        display(inst.results[:])        

One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-05-04 00:00:00,LOSS,-0.00173,0.0,1.01775,1.02467,0.00160
2017-05-09 12:00:00,PROFIT,0.00546,1.0,1.00366,1.01094,0.00161
2017-06-09 16:00:00,LOSS,-0.00118,0.0,1.01184,1.01656,0.00124
2017-06-22 19:00:00,PROFIT,0.00555,1.0,0.99687,1.00427,0.00155


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-03-09 00:00:00,LOSS,-0.00120,0.0,0.76250,0.76730,0.00115
2017-03-27 17:00:00,PROFIT,0.00420,1.0,0.74734,0.75294,0.00159
2017-04-18 16:00:00,LOSS,-0.00150,0.0,0.75329,0.75929,0.00130
2017-04-18 20:00:00,LOSS,-0.00144,1.0,0.75178,0.75754,0.00126
2017-04-26 15:00:00,LOSS,-0.00152,0.0,0.74297,0.74905,0.00139
2017-04-26 17:00:00,LOSS,-0.00171,0.0,0.74163,0.74847,0.00136
2017-05-04 19:00:00,LOSS,-0.00115,0.0,0.73060,0.73520,0.00148
2017-05-31 18:00:00,LOSS,-0.00173,1.0,0.71868,0.72560,0.00165


One month details:


...

,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-03-21 17:00:00,LOSS,-63.31,0.0,20702.99,20956.23,92.0
2017-05-11 17:00:00,PROFIT,117.93,2.0,20795.59,20952.83,39.0
2017-05-17 03:00:00,LOSS,-32.81,0.0,20818.89,20950.13,60.0


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-03-16 02:00:00,LOSS,-0.00110,0.0,1.32821,1.33261,0.00136
2017-03-16 09:00:00,PROFIT,0.00732,1.0,1.32754,1.33730,0.00264
2017-03-21 15:00:00,PROFIT,0.00558,0.0,1.32629,1.33373,0.00239
2017-04-24 13:00:00,PROFIT,0.00498,0.0,1.34098,1.34762,0.00221
2017-06-22 17:00:00,PROFIT,0.00633,1.0,1.32072,1.32916,0.00295


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-03-16 02:00:00,LOSS,-0.00114,0.0,0.99834,1.00290,0.00167
2017-03-16 08:00:00,LOSS,-0.00111,0.0,0.99810,1.00254,0.00142
2017-03-27 10:00:00,LOSS,-0.00188,0.0,0.98350,0.99102,0.00167
2017-03-27 11:00:00,LOSS,-0.00131,0.0,0.98418,0.98942,0.00122
2017-03-27 16:00:00,PROFIT,0.00639,1.0,0.98125,0.98977,0.00211
2017-05-16 14:00:00,LOSS,-0.00202,0.0,0.98908,0.99716,0.00196
2017-05-17 17:00:00,LOSS,-0.00164,1.0,0.97735,0.98391,0.00260
2017-05-31 14:00:00,LOSS,-0.00127,0.0,0.96998,0.97506,0.00123
2017-06-05 00:00:00,PROFIT,0.00669,4.0,0.96090,0.96982,0.00241


<a id="eksperyment3summary"></a>
### Podsumowanie zbiorcze - Młotki

In [57]:
posivite_instruments = 0
for iname in sorted(INSTRUMENTS.keys()):
    print('%11s   %g' % (iname, INSTRUMENTS[iname].balance))
    if INSTRUMENTS[iname].balance > 0:
        posivite_instruments += 1
print()
print("%d/%d (%.2f%%)" % (posivite_instruments, len(INSTRUMENTS),  100. * posivite_instruments / len(INSTRUMENTS)))

     AUDCAD   0.0081
     AUDCHF   -0.00605
     AUDJPY   -0.38
     AUDNZD   -0.00362
BRENT-AUG17   0.06
     CADJPY   -1.073
COCOA-SEP17   201
     EURAUD   -0.00482
     EURCHF   0
     EURDKK   -0.00685
     EURHKD   0.16509
     EURNOK   -0.11195
     EURPLN   0.05562
     EURRUB   -0.6083
     EURSEK   0.04911
     EURUSD   -0.00557
     GBPCAD   -0.01158
  GER30Cash   191.95
       GOLD   -21.49
  POL20Cash   0
     SILVER   -0.392
SUGAR-JUL17   -0.0087
   US30Cash   21.81
     USDCAD   0.02311
     USDCHF   0.00271

10/25 (40.00%)


<a id='eksperyment4'></a>
## Eksperyment 4) Analiza zbiorcza dla wszystkich walorów - Spadające gwiazdy

In [59]:
%%time

for iname in sorted(INSTRUMENTS.keys()):
    print(iname)
    inst = INSTRUMENTS[iname]
    inst.results = pd.DataFrame( columns = range(7) )

    stats = {}
    balance = 0.
    for idx, row in inst.data['2017/01/01':].iterrows():  # XXX Month
        if jestSpadajacaGwiazda(row) and zasada1120(inst,idx) and inst.rsi[idx] >= 70: # XXX jestMlotek
            res = sprawdz(inst, idx, 'sell') # XXX buy/sell
            if not res or type(res) == str:
                continue        
            inst.results = inst.results.append([ (idx,) + res ])
            if type(res) == tuple:
                balance += res[1] # XXX tu ewentualnie uwzgledniac swap * nswaps
                if res[0] not in stats:
                    stats[res[0]] = 0
                stats[res[0]] += 1
    inst.results.columns = ['date', 'PL', 'gv', 'nswaps', 'sl', 'tp', 'size']
    inst.results.set_index('date', inplace=True)
    inst.balance = balance
    inst.stats = stats
    inst.success_rate = 0
    if 'PROFIT' in stats:
        inst.success_rate = stats['PROFIT']/ sum(stats.values())

AUDCAD
AUDCHF
AUDJPY
AUDNZD
BRENT-AUG17
CADJPY
COCOA-SEP17
EURAUD
EURCHF
EURDKK
EURHKD
EURNOK
EURPLN
EURRUB
EURSEK
EURUSD
GBPCAD
GER30Cash
GOLD
POL20Cash
SILVER
SUGAR-JUL17
US30Cash
USDCAD
USDCHF
CPU times: user 24.3 s, sys: 5.59 ms, total: 24.3 s
Wall time: 24.3 s


In [60]:
for iname in sorted(INSTRUMENTS.keys()):
    inst = INSTRUMENTS[iname]
    if inst.results is not None:
        display(HTML('<h3>%s</h3>' % iname))
        display(HTML('<b>Balance: </b> %f &nbsp;&nbsp;&nbsp;&nbsp; <b>Trades:</b>%s' % (inst.balance, inst.stats)))
        display(HTML('<b>Success rate: </b> %.2f%%' % (100. * inst.success_rate)))
        print("One month details:")
        display(inst.results['2017/06/01':])        

One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-06-05 17:00:00,PROFIT,0.00456,1.0,1.01046,1.00438,0.00141


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,


One month details:


...

,PL,gv,nswaps,sl,tp,size
date,,,,,,
2017-06-19 20:00:00,LOSS,-21.61,0.0,21522.01,21435.57,20.5


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,


One month details:


,PL,gv,nswaps,sl,tp,size
date,,,,,,


<a id="eksperyment4summary"></a>
### Podsumowanie zbiorcze - spadające gwiazdy

In [61]:
posivite_instruments = 0
for iname in sorted(INSTRUMENTS.keys()):
    print('%11s   %g' % (iname, INSTRUMENTS[iname].balance))
    if INSTRUMENTS[iname].balance > 0:
        posivite_instruments += 1
print()
print("%d/%d (%.2f%%)" % (posivite_instruments, len(INSTRUMENTS),  100. * posivite_instruments / len(INSTRUMENTS)))

     AUDCAD   -0.00454
     AUDCHF   -0.00376
     AUDJPY   0.286
     AUDNZD   -0.00191
BRENT-AUG17   -0.9
     CADJPY   0.45
COCOA-SEP17   0
     EURAUD   -0.00227
     EURCHF   0.00051
     EURDKK   -0.00434
     EURHKD   -0.07229
     EURNOK   -0.17586
     EURPLN   -0.02726
     EURRUB   -4.6029
     EURSEK   0.03961
     EURUSD   0.00114
     GBPCAD   -0.02444
  GER30Cash   -40.61
       GOLD   -23.6
  POL20Cash   -40
     SILVER   -0.582
SUGAR-JUL17   0
   US30Cash   -83.33
     USDCAD   -0.00335
     USDCHF   -0.00109

5/25 (20.00%)
